In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from rossmann_toolbox import RossmannToolbox
rtb = RossmannToolbox(use_gpu=True)

Using backend: pytorch


In [3]:
models = [i.strip() for i in open('/home/nfs/sdunin/calc/DeepLigand/brute_scan/dlmodels.txt').readlines()]

In [4]:
from rossmann_toolbox.models.structmodel import GAT

In [5]:
import sys
import os
import pandas as pd
PATH_FOLDX_FLOW = '/home/users/kkaminski/DL/ewolutor/evo'

PATH_LOCALPDB_REPO = '/home/nfs/jludwiczak/localpdb'
PATH_LOCALPDB_STRUC = '/home/db/localpdb'#
LOCALPDB_PLUGINS = ['PDBSeqresMapper', 'Biounit']
PATH_CACHED_STRUCTURES = '/home/users/kkaminski/DL/cached_structures'
PATH_FOLDX = '/home/users/sdunin/apps/foldx5/foldx'
sys.path.extend([PATH_FOLDX_FLOW, PATH_LOCALPDB_REPO, PATH_LOCALPDB_STRUC])

In [6]:
#parsing structure
from Bio.SeqUtils import seq1
import atomium

In [7]:
#foldx
from modelling import tools, myfoldx

In [8]:
#localpdb
from localpdb import PDB
localpdb = None

In [9]:
#code utils
from shutil import copyfileobj
import gzip

In [31]:
# feature preparation
from rossmann_toolbox.utils import feats_from_stuct_file, separate_beta_helix

In [11]:
# DL3D flow
from rossmann_toolbox.utils import Deepligand3D

In [12]:
%ls ../data/

bak_old/
clusters_df.p
cores_df.p
data/
PC_09_TF_07_COM_1_CLM_2_EXP_DS_v12_20200717.p
split.fas
split.fas.phr
split.fas.pin
split.fas.psq
tempq.fas
tmpblasthsp.txt
TrainTestValid_TF_06_COM_0_CLM_0_DS_v12_20200717.p
TrainTestValid_TF_06_COM_0_CLM_0_NOEXP_DS_v12_20200717.p


In [13]:
df = pd.read_pickle('../data/TrainTestValid_TF_06_COM_0_CLM_0_NOEXP_DS_v12_20200717.p')

In [14]:
df[df['class'] == 'train'].pdb_chain.tolist()[:10]

['1af7_A',
 '1bsv_A',
 '1bw9_A',
 '1b4v_A',
 '1coy_A',
 '1d4f_A',
 '1d4o_A',
 '1an9_A',
 '1dhr_A',
 '1dl5_A']

In [15]:
if localpdb is None:
    localpdb = PDB(db_path=PATH_LOCALPDB_STRUC, plugins=LOCALPDB_PLUGINS)
available_structures = localpdb.structures['biounit']

In [16]:
#add some chains
dl3d_train_cores = ['1af7_A','1bsv_A','1bw9_A','1b4v_A','1coy_A','1d4f_A','1d4o_A','1an9_A','1dhr_A','1dl5_A']
dl3d_other_cores = ['1b14_A', '1dli_A']
cores_to_process = localpdb.chains.loc[localpdb.chains.index.isin(dl3d_train_cores+dl3d_other_cores), 'sequence'].to_dict()

* https://gitlab.com/labstructbioinf/rossmanndata/-/blob/master/DeepLigand/3b_Publication_Web.ipynb

In [17]:
data = {'seq1': 'MSKKFNGKVCLVTGAGGNIGLATALRLAEEGTAIALLDMNREAL', 
        'seq2': 'MSKKFNGKVCLVTGAGGNIGLATALRLAEEGTAIALLSRNREAL'}
preds = rtb.predict(data, mode='core', importance=False)
preds = {'seq1': {'FAD': 0.0010036804,
  'FAD_std': 0.001190269,
  'NAD': 0.9867387,
  'NAD_std': 0.016175654,
  'NADP': 0.014890989,
  'NADP_std': 0.015133685,
  'SAM': 0.00017169576,
  'SAM_std': 0.0002028175},
 'seq2': {'FAD': 6.141185e-08,
  'FAD_std': 5.1703925e-08,
  'NAD': 1.699253e-05,
  'NAD_std': 2.501946e-05,
  'NADP': 0.9999896,
  'NADP_std': 1.4234308e-05,
  'SAM': 1.1083341e-05,
  'SAM_std': 1.9881409e-05}}

In [18]:
data = {'1piw_A': 'MSYPEKFEGIAIQSHEDWKNPKKTKYDPKPFYDHDIDIKIEACGVCGSDIHCAAGHWGNMKMPLVVGHEIVGKVVKLGPKSNSGLKVGQRVGVGAQVFSCLECDRCKNDNEPYCTKFVTTYSQPYEDGYVSQGGYANYVRVHEHFVVPIPENIPSHLAAPLLCGGLTVYSPLVRNGCGPGKKVGIVGLGGIGSMGTLISKAMGAETYVISRSSRKREDAMKMGADHYIATLEEGDWGEKYFDTFDLIVVCASSLTDIDFNIMPKAMKVGGRIVSISIPEQHEMLSLKPYGLKAVSISYSALGSIKELNQLLKLVSEKDIKIWVETLPVGEAGVHEAFERMEKGDVRYRFTLVGYDKEFSD'}
preds = rtb.predict(data, mode='seq', importance=False)

preds = {'1piw_A': {'FAD': 2.0404239e-11,
  'FAD_std': 3.5465863e-11,
  'NAD': 7.502697e-09,
  'NAD_std': 7.654336e-09,
  'NADP': 1.0,
  'NADP_std': 0.0,
  'SAM': 8.859513e-09,
  'SAM_std': 1.7599064e-08,
  'sequence': 'NGCGPGKKVGIVGLGGIGSMGTLISKAMGAETYVISRSSRKR'}}

# utils

In [19]:
class StructureNotFound(Exception):
    pass
class ChainNotFound(Exception):
    pass

In [20]:
class StructPrep:
    path=PATH_CACHED_STRUCTURES
    foldx_suffix = '_Repair.pdb'

    def __init__(self):
        self.foldx_feat_suffix = self.foldx_suffix.replace('.pdb', '.fxout')
        self._read_cache()
        pass
    
    def find_source_struct_file(self, struct_id):
        
        if struct_id.endswith('.pdb'):
            struct_id = struc_id.replace('.pdb', '')
        
        source_struct_file = available_structures[available_structures.index == struct_id]
        if source_struct_file.empty:
            raise StructureNotFound('no structure file for ', struct_id)
        else:
            source_struct_file = source_struct_file.tolist()[0]
        print('Found structrue for %s -> %s' %(struct_id, source_struct_file))
        return source_struct_file
    
    def _read_cache(self):
        self.files = os.listdir(self.path)
        self.files_struct = [f for f in self.files if f.find(self.foldx_suffix) == -1]
        self.files_foldx = [f for f in self.files if f.find(self.foldx_suffix) != -1]
        self.files_foldx_feats = [f for f in self.files if f.find(self.foldx_feat_suffix) != -1]
          
    def is_structure_file_cached(self, file_pdb):

        condition = False
        path_file_full = os.path.join(self.path, file_pdb)
        if file_pdb in self.files_struct:
            if os.path.getsize(path_file_full) > 0:
                condition = True
        return condition
    
    def is_foldx_feat_file_cached(self, file_pdb):
        
        assert file_pdb.endswith(self.foldx_feat_suffix), 'foldx feats should match %s extension' %self.foldx_feat_suffix

        condition = False
        path_file_full = os.path.join(self.path, file_pdb)
        if file_pdb in self.files_foldx_feats:
            if os.path.getsize(path_file_full) > 0:
                condition = True
        return condition
    
    def is_foldx_file_cached(self, file_pdb):

        assert file_pdb.endswith(self.foldx_suffix), 'foldx feats should match %s extension' %self.foldx_suffix

        condition = False
        path_file_full = os.path.join(self.path, file_pdb)
        if file_pdb in self.files_foldx:
            if os.path.getsize(path_file_full) > 0:
                condition = True
        return condition
    
    def cache_struct(self, source_pdb):
        
        basename = os.path.basename(source_pdb)
        if basename.endswith('gz'):
            basename = basename.replace('.gz', '')
            
        destination_pdb = os.path.join(self.path, basename)
        with open(destination_pdb, 'wt') as f_destination:
            with gzip.open(source_pdb, 'rt') as f_source:
                copyfileobj(f_source, f_destination)
                
        return destination_pdb

struct_prep = StructPrep()

# prepare struct & foldX data

In [21]:
pdb_id, seq_full = cores_to_process.copy().popitem()
struc_id, chain_id = pdb_id.split('_')

struct_file_basename = '%s.pdb' %pdb_id
foldx_file_basename = '%s%s'%(pdb_id, struct_prep.foldx_suffix)
foldx_feats_file_basename = '%s%s'%(pdb_id, struct_prep.foldx_feat_suffix)

struct_file = os.path.join(struct_prep.path, struct_file_basename)
foldx_file = os.path.join(struct_prep.path, foldx_file_basename)
foldx_feats = os.path.join(struct_prep.path, foldx_feats_file_basename)

#check is structure is cached
if struct_prep.is_structure_file_cached(struct_file_basename):
    print('structure file %s exists' %struct_file_basename)
else:
    source_struct_file = struct_prep.find_source_struct_file(struct_file_basename)
    temp = atomium.fetch(struc_id.upper())
    temp.model.chain(chain_id.upper()).save(struct_file)
#check if foldx structure is cached 
if struct_prep.is_foldx_file_cached(foldx_file_basename):
    print('foldx file %s already exists' %foldx_file_basename)
else:
    print('creating foldx file')
    work_dir = os.getcwd()
    #change working directory to `PATH_CACHED_STRUCTURES` without that
    #foldx cant find structure error `No pdbs for the run found at: "./" Foldx will end`
    os.chdir(struct_prep.path)
    cmd = f'{PATH_FOLDX} --command=RepairPDB --pdb={struct_file_basename}'
    out = tools.run_command(cmd)
    myfoldx.fix_TER(foldx_file_basename)
if struct_prep.is_foldx_feat_file_cached(foldx_feats_file_basename):
    print('foldx feats file %s already exists' %foldx_feats_file_basename)
else:
    print('creating foldx feature file...')
    # calculate structural features using foldX 
    # it always calc features - probably some kind of file name error
    fx = myfoldx.MyFoldX(struct_prep.path, struct_prep.path, PATH_FOLDX)
    fx._MyFoldX__calc_foldx_features(struct_prep.path, foldx_file_basename)
    print('foldx file %s created' %path_foldx_struct)  
    # calculate structural features using foldX
    

structure file 1dli_A.pdb exists
foldx file 1dli_A_Repair.pdb already exists
foldx feats file 1dli_A_Repair.fxout already exists


# find cores

# create frame

In [32]:
structure = atomium.open(foldx_file)
chain = structure.model.chain(chain_id)
if chain is None:
    raise ChainNotFound('chain %s not found in struct: %s ' %(chain_id, struc_id))
    
pdb_res = [res for res in chain.residues() if seq1(res.name)!='X']
pdbids = [res.id.split('.')[1] for res in pdb_res]
pdbseq = "".join([seq1(res.name) for res in pdb_res]) 


# find core
data = {pdb_id: pdbseq}
detected_cores = rtb.seq_detect_cores(data)
filtred_cores = rtb._get_cores_from_seq(data, detected_cores)
frame_list = list()
for pdb_chain, core_seq in filtred_cores.items():
    core_pos = pdbseq.find(core_seq)
    if core_pos == -1:
        error('could not map the core onto the sequence')
    pdb_list = pdbids[core_pos:core_pos+len(core_seq)]
    ss = tools.extract_core_dssp(foldx_file, pdb_list, core_seq)
    ss = separate_beta_helix(ss)
    
    frame_list.append({
        'pdb_chain' : pdb_chain,
        'seq' : core_seq,
        'pdb_list' : pdb_list,
        'fname' : struct_file,
        'secondary' : ss
    })
    
    
frame = pd.DataFrame(frame_list)

# prepare features

In [23]:
foldx_info, distances_dict, edge_dict = feats_from_stuct_file(frame)

parsing /home/users/kkaminski/DL/cached_structures/InteractingResidues_Hbonds_Optimized_1dli_A_Repair_PN.fxout
--- Residue-Residue Interactions
--- Mainchain-Mainchain Interactions
--- Sidechain-Mainchain Interactions
--- Sidechain-Sidechain Interactions
parsing /home/users/kkaminski/DL/cached_structures/InteractingResidues_Volumetric_Optimized_1dli_A_Repair_PN.fxout
--- Residue-Residue Interactions
--- Mainchain-Mainchain Interactions
--- Sidechain-Mainchain Interactions
--- Sidechain-Sidechain Interactions
parsing /home/users/kkaminski/DL/cached_structures/InteractingResidues_Electro_Optimized_1dli_A_Repair_PN.fxout
--- Residue-Residue Interactions
--- Mainchain-Mainchain Interactions
--- Sidechain-Mainchain Interactions
--- Sidechain-Sidechain Interactions
parsing /home/users/kkaminski/DL/cached_structures/InteractingResidues_VdWClashes_Optimized_1dli_A_Repair_PN.fxout
--- Residue-Residue Interactions
--- Mainchain-Mainchain Interactions
--- Sidechain-Mainchain Interactions
--- Side

# make predictions

In [24]:
dl3d = Deepligand3D(models)

loaded 4 models


In [34]:
preds = dl3d.predict(frame, distances_dict, edge_dict, foldx_info)

seq to process: 1 1


In [35]:
preds

,NAD,NADP,SAM,FAD,cofactor,seq,NAD_std,NADP_std,SAM_std,FAD_std
0,0.875,0.625,0.25,0.5,NAD,MKIAVAGSGYVGLSLGVLLSLQNEVTIVDILPSKV,0.25,0.478714,0.5,0.57735
